In [1]:
%load_ext autoreload
%autoreload 2

import sys; sys.path.append(2*'../')

import os
import glob
from pathlib import Path
from omegaconf import DictConfig
import yaml

import torch
import lightning as L

from rl4co.tasks.rl4co import RL4COLitModule
from rl4co.utils.lightning import load_model_from_checkpoint
from rl4co.tasks.eval import evaluate_policy


/home/botu/botu/Dev/rl4co/env/lib/python3.10/site-packages/torchrl/__init__.py:26: UserWarning: failed to set start method to spawn, and current start method for mp is fork.
  warn(


## Loading model from checkpoint

In [2]:
import pickle

fpath = '../../results/eval_methods_pareto/tsp50/am-tsp50/results.pkl'

with open(fpath, 'rb') as f:
    results = pickle.load(f)

In [13]:
experiments = list(results.keys())

for exp in experiments:
    print(exp)
    num_exps = len(results[exp])
    print(num_exps)
    print(results[exp])

greedy
1
[{'actions': tensor([[44, 28, 43,  ..., 16,  8, 26],
        [ 3,  6, 48,  ..., 41,  9, 25],
        [49, 26, 10,  ..., 45, 31, 28],
        ...,
        [ 5, 33, 15,  ...,  8, 13, 35],
        [ 2,  9, 22,  ...,  5,  8, 10],
        [41, 34, 21,  ..., 48,  9, 16]]), 'rewards': tensor([-14.8449, -17.9183, -22.1559,  ..., -19.1024, -17.8643, -18.8610]), 'inference_time': 622.5745849609375, 'avg_reward': tensor(-17.3433), 'exp_name': 'greedy', 'exp_kwargs': {}}]
augment
2
[{'actions': tensor([[44, 28, 43,  ..., 16,  8, 26],
        [ 3, 15, 40,  ..., 41,  0, 39],
        [20, 40, 23,  ..., 22, 14,  4],
        ...,
        [21, 49, 24,  ...,  8, 38, 25],
        [15,  7, 45,  ..., 10,  4, 19],
        [16,  2, 17,  ...,  0,  1, 29]]), 'rewards': tensor([-15.1276, -11.0022, -12.2947,  ..., -14.9368, -17.7912, -14.0221]), 'inference_time': 3078.867431640625, 'avg_reward': tensor(-13.7893), 'exp_name': 'augment', 'exp_kwargs': {'num_augment': 8}}, {'actions': tensor([[ 3, 30, 10,  

In [21]:
results['greedy'][0]['rewards']

tensor([-14.8449, -17.9183, -22.1559,  ..., -19.1024, -17.8643, -18.8610])

In [12]:
checkpoints_path = Path('../../saved_checkpoints/')

exp_name = 'tsp50'
model_name = 'am-tsp50'
checkpoints_path = checkpoints_path / exp_name / model_name

cfg_path = checkpoints_path / 'config.yaml'
ckpt_path = checkpoints_path / 'epoch_099.ckpt'

lit_module = load_model_from_checkpoint(cfg_path, ckpt_path, phase='test')

Unused kwargs: {'params': {'total': 708608, 'trainable': 708608, 'non_trainable': 0}}


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

policy = lit_module.model.policy.to(device)
policy.eval()
env = lit_module.model.env

In [4]:
test_dataset = lit_module.test_dataset


out_aug = evaluate_policy(env, policy, test_dataset, method="augment", num_augment=20)


Effective batch size: 1024 (ratio: 20)
Using automatic batch size: 1024


Running augmentation:   0%|          | 0/10 [00:00<?, ?it/s]

Mean reward for augmentation: -5.7103
Time: 7.7164s


In [5]:
test_dataset = lit_module.test_dataset

# Greedy
out_greedy = evaluate_policy(env, policy, test_dataset, method="greedy")

# Sampling
out_samppling = evaluate_policy(env, policy, test_dataset, method="sampling", samples=50)

# Greedy multistart
out_greedy_ms = evaluate_policy(env, policy, test_dataset, method="greedy_multistart")#, num_starts=20)

# Augment dihedral 8 (same as POMO)
out_augd = evaluate_policy(env, policy, test_dataset, method="augment_dihedral_8")

# Symmetric augment
out_aug = evaluate_policy(env, policy, test_dataset, method="augment", num_augment=50)

# Greedy multistart with augment dihedral 8
out_greedy_ms_augd = evaluate_policy(env, policy, test_dataset, method="greedy_multistart_augment_dihedral_8")

# Greedy multistart with symmetric augment
out_greddy_ms_aug = evaluate_policy(env, policy, test_dataset, method="greedy_multistart_augment", num_augment=50)

Effective batch size: 4096 (ratio: 1)
Using automatic batch size: 4096


Running greedy:   0%|          | 0/3 [00:00<?, ?it/s]

Mean reward for greedy: -5.7785
Time: 0.5134s
Effective batch size: 512 (ratio: 50)
Using automatic batch size: 512


Running sampling:   0%|          | 0/20 [00:00<?, ?it/s]

Mean reward for sampling: -5.7340
Time: 18.9422s
Effective batch size: 4096 (ratio: 5)
Using automatic batch size: 4096


...

Mean reward for augmentation: -5.7048
Time: 19.1155s
Effective batch size: 512 (ratio: 40)
Using automatic batch size: 512


Running greedy_multistart_augment:   0%|          | 0/20 [00:00<?, ?it/s]

Mean reward for greedy_multistart_augment: -5.7148
Time: 14.8714s
Effective batch size: 128 (ratio: 250)
Using automatic batch size: 128


Running greedy_multistart_augment:   0%|          | 0/79 [00:00<?, ?it/s]

Mean reward for greedy_multistart_augment: -5.7023
Time: 91.0452s
